# **Auxiliary Discriminator Training with 3D Dataset**

## Prepare necessary libraries and helper functions to build dataset

In [ ]:
# ================== IMPORT LIBRARIES ==================
import os

# This allow a silent import of Tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import time
import json
from google_drive_downloader import GoogleDriveDownloader

from IPython import display

print("Successfully Load All Libraries - Tensorflow Version {}".format(tf.__version__))

In [ ]:
# Path of the directory where all the Json files are saved. 
data_files_directory = './data/chair_voxel_dataset/'

Download the dataset. 
Each file is a JSON file that contains information of the geometry we converted from the ShapeNet dataset and the related ratings. 

The JSON file should have following properties: 

*   "geometry":  a multi-dimension array to represent the geometry. 
*   "voxel_size": the size of each voxels (should always equal to 1 in the context of this notebook)
*   "scaling_factor_from_original": the scaling factor we used to scale the original mesh to produce this geometry
*   "geometry_shape": a list representing the shape of the geometry. (should always be [32, 32, 64] in the context of this notebook)

Ratings in the JSON files are as following: 

If the geometry is not given one of the ratings, the properties will note null
*   "stability_rating": the stability rating as a float from 0 to 1
*   "aesthetic_rating": the aesthetic rating as an integer from 0 to 10
*   "function_rating": the function rating as an integer from 0 to 10

Although the aesthetic_rating and function_rating are from 0 to 10 in the dataset. They should be normalized for the training, including the pretrained network should also produce a normalized rating. 

In [ ]:
# If the directory exists and is not empty we presume the dataset is already on the local drive. 
# If not, we will download the dataset. 

download_dataset_from_google_drive = False

if os.path.isdir(data_files_directory):
    if not os.listdir(data_files_directory):
        # Directory is empty
        download_dataset_from_google_drive = True
    else:    
        # Directory is not empty
        download_dataset_from_google_drive = False
else:
    # Given directory doesn't exist
    download_dataset_from_google_drive = True
    
if download_dataset_from_google_drive:

    # This will get download all the json files that contain geometry and rating information from google drive to the destination directory
    GoogleDriveDownloader.download_file_from_google_drive(file_id='1DJzSfSWUDkAhuEfu706pm0riC6ho_5NS', 
                                                          dest_path=os.path.join(data_files_directory, 'chair_voxel_dataset.zip'),
                                                          unzip=True)
    data_files_directory = data_files_directory + 'chair_voxel_dataset/'

In [ ]:
# Get all the json file paths from the unzippped directory. 
data_file_paths = glob.glob(data_files_directory + "*.json")

In [ ]:
data_files_with_stability_rating = []
data_files_with_aesthetic_rating = []
data_files_with_function_rating = []

for data_file in data_file_paths:
    with open(data_file) as json_file:
        dictionary = json.load(json_file)
        if dictionary["stability_rating"] != None:
            data_files_with_stability_rating.append(data_file)
        if dictionary["aesthetic_rating"] != None:
            data_files_with_aesthetic_rating.append(data_file)
        if dictionary["function_rating"] != None:
            data_files_with_function_rating.append(data_file)

In [ ]:
print("There are {} json files in total. ".format(len(data_file_paths)))
print("There are {} files that have stability rating. ".format(len(data_files_with_stability_rating)))
print("There are {} files that have aesthetic rating. ".format(len(data_files_with_aesthetic_rating)))
print("There are {} files that have function rating. ".format(len(data_files_with_function_rating)))

Define several helper functions below that will facilitate building the tensorflow dataset. 

In [ ]:
# ================= HELPER FUNCTIONS ===================
# This function can preview a geometry data using matplot lib
def preview_geometries(geometry_data, plot_threshold=0.25, stability_model=None, aesthetic_model=None, function_model=None, save_plot_as_png=False, path_to_save=None):
    # geometry_data must be an array of shape [32, 32, 64, 1]
    if stability_model is not None:
        stability_prediction = np.array(stability_model(np.expand_dims(geometry_data, axis=0)))[0][0]
    else:
        stability_prediction = None
    
    if aesthetic_model is not None:
        aesthetic_prediction = np.array(aesthetic_model(np.expand_dims(geometry_data, axis=0)))[0][0]
    else:
        aesthetic_prediction = None
    
    if function_model is not None:
        function_prediction = np.array(function_model(np.expand_dims(geometry_data, axis=0)))[0][0]
    else:
        function_prediction = None
    
    generation_title = "Stability Prediction is {}\nAesthetic Prediction is {}\nFunction Prediction is {}".format(str(stability_prediction), str(aesthetic_prediction), str(function_prediction))

    x = []
    y = []
    z = []

    data = np.squeeze(geometry_data, axis=-1)
    for i in range(0, len(data)):
        for j in range(0, len(data[i])):
            for k in range(0, len(data[i][j])):
                if data[i][j][k] >= plot_threshold:
                    x.append(i)
                    y.append(j)
                    z.append(k)

    z_c = z

    # We decided to plot the results with a gradient color map so the depth and geometries are clearer to see. 
    from matplotlib.colors import ListedColormap

    N = 256
    vals = np.ones((N, 4))
    vals[:, 0] = np.linspace(1, 0, N)
    vals[:, 1] = np.linspace(43/N, 43/N, N)
    vals[:, 2] = np.linspace(82/N, 82/N, N)
    custom_cmp = ListedColormap(vals)

    fig = plt.figure(figsize=(5,5))
    fig = plt.axes(projection='3d')
    
    # Data for three-dimensional scattered points
    fig.scatter3D(np.array(x), np.array(y) * -1, np.array(z), cmap=custom_cmp, c=z_c)
    fig.set_zlim(0,64)
    fig.set_ylim(-64,0)
    fig.set_xlim(0,64)
    plt.title(generation_title);
    if (save_plot_as_png):
        plt.savefig(path_to_save + '.png')
  
    plt.show()
    return plt

# This function reads a custom dictionary data
def read_json_file_geometry(filepath):
    with open(filepath.numpy()) as json_file:
        dictionary = json.load(json_file)

    tensor = tf.convert_to_tensor(np.array(dictionary["geometry"]), dtype=tf.float32)
    tensor = tf.expand_dims(tensor, -1)
    return tensor

# This function returns the value of item in custom dictionary data from given key
def read_json_file_stability(filepath):
    with open(filepath.numpy()) as json_file:
        dictionary = json.load(json_file)
    tensor = tf.convert_to_tensor(np.array(dictionary["stability_rating"]), dtype=tf.float32)
    tensor = tf.expand_dims(tensor, -1)
    return tensor

# This function returns the value of item in custom dictionary data from given key
def read_json_file_aesthetic(filepath):
    with open(filepath.numpy()) as json_file:
        dictionary = json.load(json_file)
    tensor = tf.convert_to_tensor(np.array(dictionary["aesthetic_rating"]), dtype=tf.float32) / 10
    tensor = tf.expand_dims(tensor, -1)
    return tensor

# This function returns the value of item in custom dictionary data from given key
def read_json_file_function(filepath):
    with open(filepath.numpy()) as json_file:
        dictionary = json.load(json_file)
    tensor = tf.convert_to_tensor(np.array(dictionary["function_rating"]), dtype=tf.float32) / 10
    tensor = tf.expand_dims(tensor, -1)
    return tensor

# Function to build dataset without labels
def build_dataset_w_labels(filepaths, label_name, batch_size):
    file_list = tf.data.Dataset.list_files(filepaths)
    ds_geometry = file_list.map(lambda x: tf.py_function(read_json_file_geometry, [x], Tout=tf.float32))
    ds_label = None
    if label_name == "stability_rating":
        ds_label = file_list.map(lambda x: tf.py_function(read_json_file_stability, [x], Tout=tf.float32))
        print("Building dataset with stability_rating as label")
    elif label_name == "aesthetic_rating":
        ds_label = file_list.map(lambda x: tf.py_function(read_json_file_aesthetic, [x], Tout=tf.float32))
        print("Building dataset with aesthetic_rating as label")
    elif label_name == "function_rating":
        ds_label = file_list.map(lambda x: tf.py_function(read_json_file_function, [x], Tout=tf.float32))
        print("Building dataset with function_rating as label")
    else:
        print("No corresponding label find... Building dataset without label")
    
    if ds_label != None:
        ds = tf.data.Dataset.zip((ds_geometry, ds_label))
        
    ds = ds.shuffle(4, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(4)
    return ds

## Stability auxiliary discriminator training

### Prepare dataset

In [ ]:
# ================= SET UP PARAMETERS ==================

# Epochs
epochs = 50

# Batch size
batch_size = 32

# Learning rate
learning_rate = 1e-4

# Path of the directory where the trained weights of the network will be saved. 
auxiliary_discriminator_save_directory = './data/pretrained_models/chair_stability_auxiliary_discriminator/'

# Create the directory to save the trained weights of the network. 
try:
    os.makedirs(auxiliary_discriminator_save_directory)
except FileExistsError:
    print("The directory to save trained network weights already exists")
    
# The path of log file that contains training information. 
training_log_location = './data/pretrained_models/chair_stability_auxiliary_discriminator/logs.txt'

# Create a txt file to save losses during training
try:
    log_file = open(training_log_location, "x")
    log_file.close()
except FileExistsError:
    print("Log file already exists")

In [ ]:
print("There are {} files that have stability rating. ".format(len(data_files_with_stability_rating)))

In [ ]:
# Define how many samples will be used for stability auxiliary discriminator training
num_training_samples = 4750

# Define how many samples will be used for stability auxiliary discriminator validation
num_validation_samples = 2027

In [ ]:
# Build tensorflow dataset

train_ds = build_dataset_w_labels(data_files_with_stability_rating[0:num_training_samples], "stability_rating", batch_size)
val_ds = build_dataset_w_labels(data_files_with_stability_rating[num_training_samples:num_training_samples + num_validation_samples], "stability_rating", batch_size)

In [ ]:
# We can preview the first geometry in first batch as following

for each_batch_geometry, each_batch_label in train_ds:
    preview_geometries(each_batch_geometry[0], plot_threshold=0.25, stability_model=None, aesthetic_model=None, function_model=None, save_plot_as_png=False, path_to_save=None)
    print("The label for this geometry is: {}".format(each_batch_label[0]))
    break

### Set up network and train

In [ ]:
# ================ SET UP MACHINE LEARNING MODELS =========
model = tf.keras.Sequential(
  [
    tf.keras.layers.Conv3D(16, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(32, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(64, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(128, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1), 
  ]
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),loss=tf.keras.losses.MeanSquaredError(reduction='sum'))

In [ ]:
history = model.fit(train_ds,
                    epochs=epochs, 
                    validation_data=val_ds, 
                    )

In [ ]:
model.save_weights(auxiliary_discriminator_save_directory + "epoch_{}".format(epochs))

In [ ]:
with open(training_log_location, "a") as log_file: 

    log_file.write("Batch Size for this training: {}".format(batch_size))
    log_file.write('\n')
    log_file.write("Learning Rate for this training: {}".format(learning_rate))
    
    for i in range(0, len(history.history["loss"])):
        log_file.write('\n')
        log_file.write('\n')
        log_file.write("Epoch: " + str(i))
        log_file.write('\n')
        log_file.write("Training loss: {:0.5f}".format(history.history["loss"][i]))
        log_file.write('\n')
        log_file.write("Validation loss: {:0.5f}".format(history.history["val_loss"][i]))
    
    log_file.close()

### Training assessment and model evaluation

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("Loss plot for auxiliary discriminator training")
plt.legend()

In [ ]:
def plot_discriminator_performance(labeled_dataset, sort_data = True):
    yvalues1 = []
    yvalues2 = []
    
    ds_iters = 0
    
    for each_batch_geometry, each_batch_labels in labeled_dataset:
        
        
        yvalues1 += model.predict(each_batch_geometry).squeeze(1).tolist()
        yvalues2 += np.array(each_batch_labels).squeeze(1).tolist()
        
        ds_iters += 1
        
        if ds_iters == int(num_training_samples/batch_size):
            break
    
    zippedarray = sorted(zip(yvalues2, yvalues1)) #sorts both lists according to the first one (xvalues2 will be the labels)
    sorted_yvalues2 = [x for x,y in zippedarray]
    sorted_yvalues1 = [y for x,y in zippedarray]

    #Plot 
    xvalues = range(0,len(yvalues1))
    fig = plt.figure()

    if (sort_data == True):
        fig1 = plt.plot(xvalues, sorted_yvalues1, label='auxiliary_discriminator_predictions')
        fig2 = plt.plot(xvalues, sorted_yvalues2, label='real_labels')

    else:
        fig1 = plt.plot(xvalues, yvalues1, label='auxiliary_discriminator_predictions')
        fig2 = plt.plot(xvalues, yvalues2, label='real_labels')

    plt.ylabel('rating')
    plt.xlabel('data no.')  
    plt.legend(loc='upper right')
    plt.title("Auxiliary Discriminator Performance")
    plt.show()

In [ ]:
plot_discriminator_performance(val_ds, sort_data = True)

In [ ]:
plot_discriminator_performance(train_ds, sort_data = True)

## Aesthetic auxiliary discriminator training

### Prepare dataset

In [ ]:
# ================= SET UP PARAMETERS ==================

# Epochs
epochs = 100

# Batch size
batch_size = 32

# Learning rate
learning_rate = 1e-5

# Path of the directory where the trained weights of the network will be saved. 
auxiliary_discriminator_save_directory = './data/pretrained_models/chair_aesthetic_auxiliary_discriminator/'

# Create the directory to save the trained weights of the network. 
try:
    os.makedirs(auxiliary_discriminator_save_directory)
except FileExistsError:
    print("The directory to save trained network weights already exists")
    
# The path of log file that contains training information. 
training_log_location = './data/pretrained_models/chair_aesthetic_auxiliary_discriminator/logs.txt'

# Create a txt file to save losses during training
try:
    log_file = open(training_log_location, "x")
    log_file.close()
except FileExistsError:
    print("Log file already exists")

In [ ]:
print("There are {} files that have aesthetic rating. ".format(len(data_files_with_aesthetic_rating)))

In [ ]:
# Define how many samples will be used for stability auxiliary discriminator training
num_training_samples = 1910

# Define how many samples will be used for stability auxiliary discriminator validation
num_validation_samples = 814

In [ ]:
# Build tensorflow dataset

train_ds = build_dataset_w_labels(data_files_with_aesthetic_rating[0:num_training_samples], "aesthetic_rating", batch_size)
val_ds = build_dataset_w_labels(data_files_with_aesthetic_rating[num_training_samples:num_training_samples + num_validation_samples], "aesthetic_rating", batch_size)

In [ ]:
# We can preview the first geometry in first batch as following

for each_batch_geometry, each_batch_label in train_ds:
    preview_geometries(each_batch_geometry[0], plot_threshold=0.25, stability_model=None, aesthetic_model=None, function_model=None, save_plot_as_png=False, path_to_save=None)
    print("The label for this geometry is: {}".format(each_batch_label[0]))
    break

### Set up network and train

In [ ]:
# ================ SET UP MACHINE LEARNING MODELS =========
model = tf.keras.Sequential(
  [
    tf.keras.layers.Conv3D(16, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(32, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(64, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Conv3D(128, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1), 
  ]
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),loss=tf.keras.losses.MeanSquaredError(reduction='sum'))

In [ ]:
history = model.fit(train_ds,
                    epochs=epochs, 
                    validation_data=val_ds, 
                    )

In [ ]:
model.save_weights(auxiliary_discriminator_save_directory + "epoch_{}".format(epochs))

In [ ]:
with open(training_log_location, "a") as log_file: 

    log_file.write("Batch Size for this training: {}".format(batch_size))
    log_file.write('\n')
    log_file.write("Learning Rate for this training: {}".format(learning_rate))
    
    for i in range(0, len(history.history["loss"])):
        log_file.write('\n')
        log_file.write('\n')
        log_file.write("Epoch: " + str(i))
        log_file.write('\n')
        log_file.write("Training loss: {:0.5f}".format(history.history["loss"][i]))
        log_file.write('\n')
        log_file.write("Validation loss: {:0.5f}".format(history.history["val_loss"][i]))
    
    log_file.close()

### Training assessment and model evaluation

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("Loss plot for auxiliary discriminator training")
plt.legend()

In [ ]:
def plot_discriminator_performance(labeled_dataset, sort_data = True):
    yvalues1 = []
    yvalues2 = []
    
    ds_iters = 0
    
    for each_batch_geometry, each_batch_labels in labeled_dataset:
        
        
        yvalues1 += model.predict(each_batch_geometry).squeeze(1).tolist()
        yvalues2 += np.array(each_batch_labels).squeeze(1).tolist()
        
        ds_iters += 1
        
        if ds_iters == int(num_training_samples/batch_size):
            break
    
    zippedarray = sorted(zip(yvalues2, yvalues1)) #sorts both lists according to the first one (xvalues2 will be the labels)
    sorted_yvalues2 = [x for x,y in zippedarray]
    sorted_yvalues1 = [y for x,y in zippedarray]

    #Plot 
    plt.style.use('default')
    text_color = 'black'
    xvalues = range(0,len(yvalues1))
    fig = plt.figure()

    if (sort_data == True):
        fig1 = plt.plot(xvalues, sorted_yvalues1, label='auxiliary_discriminator_predictions')
        fig2 = plt.plot(xvalues, sorted_yvalues2, label='real_labels')

    else:
        fig1 = plt.plot(xvalues, yvalues1, label='auxiliary_discriminator_predictions')
        fig2 = plt.plot(xvalues, yvalues2, label='real_labels')

    plt.ylabel('Rating', color=text_color)
    plt.xlabel('Data no.', color=text_color)  
    plt.rc_context({'xtick.color' : text_color})
    plt.legend(loc='upper right')
    plt.title("Auxiliary Discriminator Performance", color=text_color)
    #plt.rcParams.update(colors('white'))
    plt.show()

In [ ]:
plot_discriminator_performance(val_ds, sort_data = True)

In [ ]:
plot_discriminator_performance(train_ds, sort_data = True)

## Function auxiliary discriminator training

### Prepare dataset

In [ ]:
# ================= SET UP PARAMETERS ==================

# Epochs
epochs = 5

# Batch size
batch_size = 32

# Learning rate
learning_rate = 1e-5

# Path of the directory where the trained weights of the network will be saved. 
auxiliary_discriminator_save_directory = './data/pretrained_models/chair_function_auxiliary_discriminator/'

# Create the directory to save the trained weights of the network. 
try:
    os.makedirs(auxiliary_discriminator_save_directory)
except FileExistsError:
    print("The directory to save trained network weights already exists")
    
# The path of log file that contains training information. 
training_log_location = './data/pretrained_models/chair_function_auxiliary_discriminator/logs.txt'

# Create a txt file to save losses during training
try:
    log_file = open(training_log_location, "x")
    log_file.close()
except FileExistsError:
    print("Log file already exists")

In [ ]:
print("There are {} files that have function rating. ".format(len(data_files_with_function_rating)))

In [ ]:
# Define how many samples will be used for stability auxiliary discriminator training
num_training_samples = 1400

# Define how many samples will be used for stability auxiliary discriminator validation
num_validation_samples = 600

In [ ]:
# Build tensorflow dataset

train_ds = build_dataset_w_labels(data_files_with_function_rating[0:num_training_samples], "function_rating", batch_size)
val_ds = build_dataset_w_labels(data_files_with_function_rating[num_training_samples:num_training_samples + num_validation_samples], "function_rating", batch_size)

In [ ]:
# We can preview the first geometry in first batch as following

for each_batch_geometry, each_batch_label in train_ds:
    preview_geometries(each_batch_geometry[0], plot_threshold=0.25, stability_model=None, aesthetic_model=None, function_model=None, save_plot_as_png=False, path_to_save=None)
    print("The label for this geometry is: {}".format(each_batch_label[0]))
    break

### Set up network and train

In [ ]:
# ================ SET UP MACHINE LEARNING MODELS =========
model = tf.keras.Sequential(
  [
    tf.keras.layers.Conv3D(16, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=(32, 32, 64, 1)), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 
   
    tf.keras.layers.Conv3D(32, (4, 4, 4), strides=(2, 2, 2), padding='same'), 
    tf.keras.layers.LeakyReLU(), 
    tf.keras.layers.Dropout(0.3), 
   
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1), 
  ]
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),loss=tf.keras.losses.MeanSquaredError(reduction='sum'))

In [ ]:
history = model.fit(train_ds,
                    epochs=epochs, 
                    validation_data=val_ds, 
                    )

In [ ]:
model.save_weights(auxiliary_discriminator_save_directory + "epoch_{}".format(epochs))

In [ ]:
with open(training_log_location, "a") as log_file: 

    log_file.write("Batch Size for this training: {}".format(batch_size))
    log_file.write('\n')
    log_file.write("Learning Rate for this training: {}".format(learning_rate))
    
    for i in range(0, len(history.history["loss"])):
        log_file.write('\n')
        log_file.write('\n')
        log_file.write("Epoch: " + str(i))
        log_file.write('\n')
        log_file.write("Training loss: {:0.5f}".format(history.history["loss"][i]))
        log_file.write('\n')
        log_file.write("Validation loss: {:0.5f}".format(history.history["val_loss"][i]))
    
    log_file.close()

### Training assessment and model evaluation

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("Loss plot for auxiliary discriminator training")
plt.legend()

In [ ]:
def plot_discriminator_performance(labeled_dataset, sort_data = True):
    yvalues1 = []
    yvalues2 = []
    
    ds_iters = 0
    
    for each_batch_geometry, each_batch_labels in labeled_dataset:
        
        
        yvalues1 += model.predict(each_batch_geometry).squeeze(1).tolist()
        yvalues2 += np.array(each_batch_labels).squeeze(1).tolist()
        
        ds_iters += 1
        
        if ds_iters == int(num_training_samples/batch_size):
            break
    
    zippedarray = sorted(zip(yvalues2, yvalues1)) #sorts both lists according to the first one (xvalues2 will be the labels)
    sorted_yvalues2 = [x for x,y in zippedarray]
    sorted_yvalues1 = [y for x,y in zippedarray]

    #Plot 
    plt.style.use('default')
    text_color = 'black'
    xvalues = range(0,len(yvalues1))
    fig = plt.figure()

    if (sort_data == True):
        fig1 = plt.plot(xvalues, sorted_yvalues1, label='auxiliary_discriminator_predictions')
        fig2 = plt.plot(xvalues, sorted_yvalues2, label='real_labels')

    else:
        fig1 = plt.plot(xvalues, yvalues1, label='auxiliary_discriminator_predictions')
        fig2 = plt.plot(xvalues, yvalues2, label='real_labels')

    plt.ylabel('Rating', color=text_color)
    plt.xlabel('Data no.', color=text_color)  
    plt.rc_context({'xtick.color' : text_color})
    plt.legend(loc='upper right')
    plt.title("Auxiliary Discriminator Performance", color=text_color)
    #plt.rcParams.update(colors('white'))
    plt.show()

In [ ]:
plot_discriminator_performance(val_ds, sort_data = True)

In [ ]:
plot_discriminator_performance(train_ds, sort_data = True)